In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from datetime import datetime
import os

2023-12-02 20:56:13.219564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 20:56:13.859953: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
DATA_PATH = Path("/scratch/ajk4yq/ecg/tfrecords/")
TEST_RECS = list(DATA_PATH.glob("test*.tfrecords"))

In [3]:
BATCH_SIZE = 8

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
    'hospital_expire_flag': tf.io.FixedLenFeature([], tf.float32)
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return ecg_data, label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [5]:
test_dataset = get_dataset(TEST_RECS)

2023-12-02 20:56:25.864709: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78919 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [6]:
tform_model = tf.keras.models.load_model("data/models/transformer-age2-2023-11-26_21-33-04")

In [7]:
test_dataset.cardinality().numpy()

-2

In [19]:
preds=tform_model.predict(test_dataset)

19045/19045 [==============================] - 2372s 125ms/step


In [21]:
preds[:50]

array([[63.14621 ],
       [73.340034],
       [67.34243 ],
       [62.81424 ],
       [71.58188 ],
       [68.42313 ],
       [77.6653  ],
       [56.88105 ],
       [86.121895],
       [47.363174],
       [64.31592 ],
       [63.711594],
       [65.059616],
       [64.31142 ],
       [86.321106],
       [44.48825 ],
       [71.24527 ],
       [45.09394 ],
       [68.731064],
       [44.36816 ],
       [50.43182 ],
       [56.95024 ],
       [44.74236 ],
       [75.04602 ],
       [59.253418],
       [44.36816 ],
       [63.38672 ],
       [46.14617 ],
       [71.64195 ],
       [68.76886 ],
       [69.46737 ],
       [49.045113],
       [66.      ],
       [60.22925 ],
       [63.036522],
       [75.677444],
       [53.17782 ],
       [76.009964],
       [80.64572 ],
       [62.29065 ],
       [73.93593 ],
       [54.96582 ],
       [58.006958],
       [74.888535],
       [69.62785 ],
       [68.37863 ],
       [69.070564],
       [66.92908 ],
       [69.385704],
       [80.56488 ]],